In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt 
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import cv2 as cv
import os 
import math
import numpy as np
from shutil import rmtree
from time import perf_counter

In [ ]:
os.listdir() 

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

In [ ]:
def generate_model(shape):
  cnn_model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(250, (3, 3), activation = "relu", input_shape = shape), #ConvLayer
                                      tf.keras.layers.MaxPool2D(2, 2), 
                                      
                                      tf.keras.layers.Conv2D(300, (4, 4), activation = "relu"),
                                      tf.keras.layers.MaxPool2D(2, 2),
                                      
                                      tf.keras.layers.Conv2D(500, (4, 4), activation = "relu"),
                                      tf.keras.layers.MaxPool2D(2, 2), 
                                      
                                      tf.keras.layers.Conv2D(256, (2,2), activation ="relu", kernel_regularizer='l1_l2', bias_regularizer="l1", activity_regularizer="l2"),

                                      tf.keras.layers.Conv2D(800, (3, 3), activation = "relu"),    # This 
                                      tf.keras.layers.MaxPool2D(2, 2),                             # This 

                                      tf.keras.layers.Conv2D(256, (2, 2), activation = "relu"),
                                      tf.keras.layers.MaxPool2D(2, 2), 

                                      tf.keras.layers.Conv2D(512, (3, 3), activation = "relu"),    # This 
                                      tf.keras.layers.MaxPool2D(2, 2),                             # This  

                                      tf.keras.layers.BatchNormalization(),

                                      tf.keras.layers.Flatten(),
                                      
                                      tf.keras.layers.Dense(500, activation = "relu"),             # This 

                                      tf.keras.layers.Dense(250, activation = "relu"),

                                      tf.keras.layers.Dense(100, activation = "relu", kernel_regularizer='l1_l2', bias_regularizer="l1_l2", activity_regularizer="l1_l2"),

                                      tf.keras.layers.BatchNormalization(),
                                      
                                      # FINAL DENSE LAYER NEURON COUNT SHOULD BE EQUAL TO OUR CLASSESS (n = 8)
                                      tf.keras.layers.Dense(8, activation = "softmax"), 
                                      
  ])

  return cnn_model

In [ ]:
def create_directories(n):
  rmtree(f"{os.path.abspath(os.getcwd())}/folds")  # Delete first
  os.mkdir(f"{os.path.abspath(os.getcwd())}/folds") # Recreate Folder
  for i in range(n):
    dir_str = f"{os.path.abspath(os.getcwd())}/folds/fold{i}"
    os.mkdir(dir_str)
    os.mkdir(f"{dir_str}/train")
    create_class_directories(f"{dir_str}/train")
    os.mkdir(f"{dir_str}/test")
    create_class_directories(f"{dir_str}/test")
    print("Directory Created!")

In [ ]:
def create_model_directories(n):
  rmtree(f"{os.path.abspath(os.getcwd())}/models_kfold")  # Delete first
  os.mkdir(f"{os.path.abspath(os.getcwd())}/models_kfold") # Recreate Folder
  for i in range(n):
     dir_str = f"{os.path.abspath(os.getcwd())}/models_kfold/fold{i}"
     os.mkdir(dir_str)

  print("Model Directories Created!")

In [ ]:
def create_class_directories(dir_str): # With the assumption that current directory is empty
  # Create correct directories
  os.mkdir(f"{dir_str}/c_bot_1")
  os.mkdir(f"{dir_str}/c_mid_1")
  os.mkdir(f"{dir_str}/c_top_1")
  os.mkdir(f"{dir_str}/c_top_2")

  # Create incorrect directories
  os.mkdir(f"{dir_str}/i_bot_1")
  os.mkdir(f"{dir_str}/i_mid_1")
  os.mkdir(f"{dir_str}/i_top_1")
  os.mkdir(f"{dir_str}/i_top_2")

  print("Classes Created!")

In [ ]:
# img_shape = cv.imread(f"{os.path.abspath(os.getcwd())}/test/c_bot_1/Adrian2_11.png").shape
# img_shape
#img_height, img_width, img_channel = cv.imread(f'{os.path.abspath(os.getcwd())}/train/i_top_2/Yen_11.png').shape
#img_shape = (img_height, img_width, img_channel)

#tmp_height = math.floor(img_height/4)
#tmp_width = math.floor(img_width/4)
shape_hyperparam_dir = (224, 224)
shape_hyperparam_cnn = (224, 224, 3) # Channel is unchanged

In [ ]:
BATCH_SIZE = 2**3
EPOCHS = 150
N_SPLITS = 10

main_images = ImageDataGenerator(rescale = 1/255)
main_dataset = main_images.flow_from_directory(f'{os.path.abspath(os.getcwd())}/100',
                                          target_size = shape_hyperparam_dir,
                                          batch_size = 1,
                                          class_mode = "categorical"
                                         ) 

# IMPORTANT NOTE: Set batch size to 1 since we'll pass the whole dataset to KFold
# **Higher batch size is better but slower ONLY DURING TRAINING!!!!**

SKFold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42) # random_state=42 means no randomness is applied
create_directories(N_SPLITS)
create_model_directories(N_SPLITS)
main_dataset.class_indices

In [ ]:
main_dataset.filenames[0]
main_dataset.filepaths[0]

In [ ]:
%%time
# VERY IMPORTANT NOTE: Always reset the directories before running the model. Use create_directories() and create_model_directories().

start = perf_counter()
scores = []
fold_counter = 0
for train_indices, test_indices in SKFold.split(main_dataset, main_dataset.classes):
  if fold_counter <= 0:
    fold_counter += 1
    continue

  # Append images to corresponding fold folder
  # NOTE: main_dataset.filenames include their classe! FORMAT: <class_name>/<filename> e.g. c_bot_1/Adrian2_04.png
  i = 1
  for train_index in train_indices:
    print(f"i count: {i}, fold: {fold_counter} path: '{os.path.abspath(os.getcwd())}/folds/fold{fold_counter}/train/{main_dataset.filenames[train_index]}")
    cv.imwrite(f'{os.path.abspath(os.getcwd())}/folds/fold{fold_counter}/train/{main_dataset.filenames[train_index]}', 
               cv.imread(f'{os.path.abspath(os.getcwd())}/100/{main_dataset.filenames[train_index]}')
               )
    i += 1

  j = 1
  for test_index in test_indices:
    print(f"j count: {j}, fold: {fold_counter} path: '{os.path.abspath(os.getcwd())}/folds/fold{fold_counter}/test/{main_dataset.filenames[test_index]}")
    cv.imwrite(f'{os.path.abspath(os.getcwd())}/folds/fold{fold_counter}/test/{main_dataset.filenames[test_index]}',
               cv.imread(f'{os.path.abspath(os.getcwd())}/100/{main_dataset.filenames[test_index]}')    
              )
    j += 1

  
  model = generate_model(shape_hyperparam_cnn)
  model.compile(loss = "categorical_crossentropy",
              optimizer = Adam(learning_rate = 0.0001),
              metrics=['accuracy']
  )

  train_images = ImageDataGenerator(rescale = 1/255)
  train_dataset = train_images.flow_from_directory(f'{os.path.abspath(os.getcwd())}/folds/fold{fold_counter}/train',
                                          target_size = shape_hyperparam_dir,
                                          batch_size = BATCH_SIZE,
                                          class_mode = "categorical"
                                         )
  
  test_images = ImageDataGenerator(rescale = 1/255)
  test_dataset = test_images.flow_from_directory(f'{os.path.abspath(os.getcwd())}/folds/fold{fold_counter}/test',
                                          target_size = shape_hyperparam_dir,
                                          batch_size = BATCH_SIZE,
                                          class_mode = "categorical"
                                         )

  history = model.fit(train_dataset,
          epochs = EPOCHS,
          validation_data = test_dataset,

  )
  end = perf_counter()
  print("----------------------TRAINING FINISHED------------")
  print(f"The model took {end - start} secs")

  path_to_model = f'{os.path.abspath(os.getcwd())}/models_kfold/fold{fold_counter}'
  scores.append(history.history['accuracy'][-1])

  model.save(f'{path_to_model}/fold{fold_counter}_model.h5')

  hist_file = open(f'{path_to_model}/fold{fold_counter}_model_latestaccu.txt', 'a')
  hist_file.write(f"latest_accuracy: {history.history['accuracy'][-1]}")
  hist_file.close()

  hist_file2 = open(f'{path_to_model}/fold{fold_counter}_model_history.txt', 'a')
  hist_file2.write(f"{history.history}")
  hist_file2.close()
  
  for metric in history.history:
      x_label = "Epoch"
      y_label = ""
      title = ""
      color = "green"

      if metric == "loss":
        title = "Training Loss"
        y_label = "Loss"
        color = "blue"

      elif metric ==  "accuracy":
        title = "Training Accuracy"
        y_label = "Accuracy"
        color = "blue"

      elif metric ==  "val_loss":
        title = "Validation Loss"
        y_label = "Loss"
        color = "red"

      elif metric ==  "val_accuracy":
        title = "Validation Accuracy"
        y_label = "Accuracy"
        color = "red"

      else:
        title = metric
        y_label = metric


      plt.plot(history.history[metric], color = color)
      plt.xlabel(x_label)
      plt.ylabel(y_label)
      plt.title(title)
      plt.savefig(f'{path_to_model}/fold{fold_counter}_{metric}.png')
      plt.show()

  fold_counter += 1


In [ ]:
print(f" Elapsed Time = {math.floor((end-start)/3600) } hours and {math.floor((end-start)/60)%60} mins")